In [1]:
import numpy as np
import pandas as pd
import statistics as st
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

import warnings
import math

from tabulate import tabulate
from typing import Union
from datetime import datetime

from Utils import *
from FinancialMetrics import *
from TitleOutOfMarket import *

## Settings

In [2]:
warnings.filterwarnings('ignore')

## Constants
The CSV files must be in the same directory of the notebook

In [3]:
ROLLING_WINDOW_SIZE = 180

CLOSING_PRICE_CSV_NAME = 'data/NASDAQ-100-CLOSING-PRICES.csv'
TBILL_CSV_NAME = 'data/13WEEKTBILLCOUPON.csv' 
CLOSING_PRICE_OF_REMOVED_CSV = 'data/CLOSING_PRICES_OF_REMOVED_TITLES.csv' 

## Data Pre-Processing
INSERIRE DESCRIZIONE
CA, ESRX, ALXN, XLNX, DISCK, TFCFA, TFCF, CELG

### Load the Closing Price Dataset

In [4]:
# Historical closing prices (with title swap)

closing_prices = pd.read_csv(CLOSING_PRICE_CSV_NAME)
closing_prices.head()

,Dates,NDX Index,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,ALGN,...,MXIM,CDW,CERN,CHKP,FOXA,FOX,INCY,TCOM,PTON,XLNX
0,3/1/2017,4911.33,29.038,NaN,103.48,72.505,103.50,76.18,NaN,NaN,...,38.62,NaN,49.40,84.57,NaN,NaN,102.320,40.49,NaN,59.07
1,4/1/2017,4937.21,29.005,NaN,104.14,72.360,103.66,77.52,NaN,NaN,...,39.03,NaN,47.93,84.92,NaN,NaN,102.840,41.34,NaN,58.64
2,5/1/2017,4964.95,29.153,NaN,105.91,71.320,103.04,76.93,NaN,NaN,...,38.44,NaN,47.65,86.66,NaN,NaN,104.810,42.80,NaN,57.93
3,6/1/2017,5007.08,29.478,NaN,108.30,71.600,103.11,79.30,NaN,NaN,...,39.74,NaN,47.55,87.24,NaN,NaN,108.305,42.50,NaN,59.05
4,9/1/2017,5024.90,29.748,NaN,108.57,71.940,102.47,79.59,NaN,NaN,...,40.65,NaN,47.99,87.20,NaN,NaN,118.525,43.24,NaN,59.06


### Calculate the log returns

In [5]:
nasdaq100_returns = get_log_returns(CLOSING_PRICE_CSV_NAME)
nasdaq100_returns.fillna(np.nan, inplace=True)
nasdaq100_returns.head()
nasdaq100_returns.head().style.applymap(color_negative_red)

,Dates,NDX Index,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,AMGN,AMZN,ANSS,ASML,ATVI,AVGO,AZN,BIDU,BIIB,BKNG,CDNS,CEG,CHTR,CMCSA,COST,CPRT,CRWD,CSCO,CSX,CTAS,CTSH,DDOG,DLTR,DOCU,DXCM,EA,EBAY,EXC,FAST,FB,FISV,FTNT,GILD,GOOG,GOOGL,HON,IDXX,ILMN,INTC,INTU,ISRG,JD,KDP,KHC,KLAC,LCID,LRCX,LULU,MAR,MCHP,MDLZ,MELI,MNST,MRNA,MRVL,MSFT,MTCH,MU,NFLX,NTES,NVDA,NXPI,ODFL,OKTA,ORLY,PANW,PAYX,PCAR,PDD,PEP,PYPL,QCOM,REGN,ROST,SBUX,SGEN,SIRI,SNPS,SPLK,SWKS,TEAM,TMUS,TSLA,TXN,VRSK,VRSN,VRTX,WBA,WDAY,XEL,ZM,ZS,SBAC,WYNN,TRIP,YHOO,MAT,TTWO,AKAM,DISCA,DISCK,NCLH,TSCO,VIAB,DISH,CA,XRAY,ESRX,HOLX,QRTEA,STX,SHPG,VOD,TFCF,TFCFA,CELG,HAS,HSIC,JBHT,VTRS,NLOK,AAL,WTW,UAL,CSGP,NTAP,WDC,BMRN,CTXS,EXPE,LBTYA,ULTA,ALXN,MXIM,CDW,CERN,CHKP,FOXA,FOX,INCY,TCOM,PTON,XLNX
1,4/1/2017,0.005256,-0.001137,nan,0.006358,-0.002002,0.001545,0.017437,nan,nan,0.009349,nan,0.014098,0.004646,nan,nan,0.019460,-0.007147,nan,0.021514,0.007992,0.006557,nan,nan,0.020024,0.011805,0.000188,nan,nan,-0.014512,0.020412,0.008475,0.014055,nan,0.025495,nan,nan,0.018273,-0.002685,nan,0.004675,0.015538,0.012130,nan,0.029496,0.000966,-0.000297,nan,nan,0.030418,-0.005205,0.002002,-0.019356,0.001161,nan,-0.005729,-0.003532,nan,0.000559,nan,-0.001941,-0.000470,-0.001115,nan,0.010485,nan,nan,-0.004484,nan,-0.008461,0.014948,0.002472,0.023063,nan,nan,nan,0.004583,nan,0.003760,0.013684,nan,nan,0.018462,0.001070,0.008941,0.014225,0.011496,nan,0.020068,nan,nan,-0.002131,nan,0.008536,0.045055,-0.001224,0.007839,nan,0.028434,0.000241,nan,nan,nan,nan,nan,nan,0.028019,0.029384,0.027019,nan,0.011185,0.011733,0.009455,0.045234,0.011658,0.019311,0.011659,0.015976,0.008155,0.024763,-0.002993,0.010947,-0.012700,0.021027,0.022580,0.008889,0.010392,-0.001686,0.014185,0.003968,nan,0.013208,0.008708,0.008602,nan,nan,nan,nan,0.015861,0.011928,0.001552,0.011495,0.022582,0.016384,0.035881,0.010560,nan,-0.030209,0.004130,nan,nan,0.005069,0.020776,nan,-0.007306
2,5/1/2017,0.005603,0.005090,nan,0.016854,-0.014477,-0.005999,-0.007640,nan,nan,-0.004352,nan,0.000719,0.030270,nan,nan,0.015405,-0.015882,nan,0.031540,-0.001633,0.010973,nan,nan,0.015722,0.003999,0.019525,nan,nan,0.002323,0.009780,-0.008819,-0.004021,nan,-0.017650,nan,nan,-0.020929,0.008365,nan,-0.011728,0.016544,-0.000368,nan,-0.004856,0.009007,0.006478,nan,nan,-0.004597,-0.001649,0.006156,0.005418,0.017258,nan,-0.005762,-0.009013,nan,-0.002239,nan,-0.006824,-0.019461,0.004454,nan,0.003687,nan,nan,0.000000,nan,-0.011244,0.018376,0.046938,-0.025713,nan,nan,nan,0.002478,nan,-0.007205,-0.016444,nan,nan,0.001462,0.001221,-0.005733,-0.013007,0.008359,nan,0.023992,nan,nan,-0.005616,nan,-0.020785,-0.001058,-0.007786,-0.003667,nan,0.025752,0.000602,nan,nan,nan,nan,nan,nan,0.006935,0.031452,0.055828,nan,0.017697,-0.039291,-0.037991,-0.004926,-0.015221,0.023759,0.006108,0.008196,-0.013114,-0.003803,-0.006014,-0.023537,-0.001791,0.012495,0.029715,0.003181,-0.000689,0.001770,0.033650,-0.005376,nan,-0.011931,0.017598,-0.017497,nan,nan,nan,nan,0.000142,-0.006699,-0.000333,0.008318,0.030472,0.012265,0.090715,-0.015232,nan,-0.005859,0.020283,nan,nan,0.018975,0.034708,nan,-0.012182
3,6/1/2017,0.008450,0.011086,nan,0.022315,0.003918,0.000679,0.030342,nan,nan,-0.001871,nan,0.024536,0.019716,nan,nan,-0.000791,0.013167,nan,-0.006161,0.004859,0.011157,nan,nan,0.006697,0.001709,-0.000491,nan,nan,0.001987,0.014229,0.003605,0.013570,nan,-0.012504,nan,nan,0.000633,0.034068,nan,-0.002362,0.022453,0.011170,nan,-0.006865,0.015161,0.014882,nan,nan,0.050219,0.003570,0.010062,0.005050,-0.001141,nan,-0.002546,0.009013,nan,0.004844,nan,0.006338,0.017265,0.001332,nan,-0.012417,nan,nan,0.008630,nan,-0.003171,-0.005630,-0.009865,0.013279,nan,nan,nan,-0.003649,nan,0.006879,0.011296,nan,nan,0.009453,-0.000305,-0.060159,-0.005802,0.011797,nan,-0.017392,nan,nan,0.005216,nan,-0.014689,0.009918,0.016727,0.008778,nan,0.004165,0.000843,nan,nan,nan,nan,nan,nan,0.031412,-0.002664,-0.015307,nan,0.003445,0.010241,0.009727,-0.006757,-0.006234,0.008237,0.009826,0.003320,-0.000686,-0.001695,0.002259,-0.004

### Load the Closing Price Dataset of stock removed from the Nasdaq-100 Index

In [6]:
# Historical prices of removed titles

closing_prices_removed_titles = pd.read_csv(CLOSING_PRICE_OF_REMOVED_CSV)
closing_prices_removed_titles.head()

,Dates,NDX Index,NXPI,IDXX,SBAC,WYNN,TRIP,MELI,YHOO,ALGN,...,MXIM,CDW,CERN,CHKP,FOXA,FOX,INCY,TCOM,PTON,XLNX
0,1/3/2017,4911.33,97.66,117.60,105.53,87.46,47.51,161.02,NaN,96.96,...,38.62,51.54,49.40,84.57,NaN,NaN,102.320,40.49,NaN,59.07
1,1/4/2017,4937.21,98.30,115.95,105.68,90.28,48.86,165.82,NaN,97.77,...,39.03,52.38,47.93,84.92,NaN,NaN,102.840,41.34,NaN,58.64
2,1/5/2017,4964.95,98.40,115.96,104.28,91.44,49.20,171.28,NaN,94.82,...,38.44,51.87,47.65,86.66,NaN,NaN,104.810,42.80,NaN,57.93
3,1/6/2017,5007.08,98.21,118.36,103.52,92.43,50.77,172.16,NaN,94.08,...,39.74,51.69,47.55,87.24,NaN,NaN,108.305,42.50,NaN,59.05
4,1/9/2017,5024.90,98.11,119.02,102.99,92.75,50.92,174.11,NaN,94.79,...,40.65,51.17,47.99,87.20,NaN,NaN,118.525,43.24,NaN,59.06


### Loading the Risk Free Asset (US Treasury Bill - 3 Months)
We've used the Coupon Equivalent. The Coupon Equivalent, also called the Bond Equivalent, or the Investment Yield, is the bill's yield based on the purchase price, discount, and a 365- or 366-day year. 

In [7]:
tbill = pd.read_csv(TBILL_CSV_NAME)
tbill['DATE'] = pd.to_datetime(tbill['DATE'])
tbill = tbill.sort_values(by='DATE', ascending=True)
tbill = tbill.reset_index().drop(['index'], axis=1)
risk_free_rate = tbill['13 WEEKS COUPON EQUIVALENT'].mean()
print(f'Annual risk-free rate: {np.round(risk_free_rate, 2)}%')

Annual risk-free rate: 1.04%


## Rolling Linear Regression
Creiamo un dataframe (rollingDataFrame) contenente le prime ROLLING_WINDOW_SIZE righe del dataframe indexes e creiamo una funzione che, a seconda dei parametri, costruisce portfoli basati su alpha, beta, r^2 o errore.
All'interno della lista indexNames inseriamo gli indici presenti in questi primi ROLLING_WINDOW_SIZE giorni.

    • rollingDataFrame: contiene le prime ROLLING_WINDOW_SIZE righe del dataframe indexes;
    • indexNames: contiene gli indici sempre presenti all'interno del Nasdaq nei primi ROLLING_WINDOW_SIZE giorni
    • selector: str in ['alpha', 'beta', 'r2', 'error']

CAMBIARE QUESTA DESCRIZIONE

In [8]:
def get_window_returns(days):
    window_returns = nasdaq100_returns.iloc[days: days + ROLLING_WINDOW_SIZE]
    window_returns = window_returns.reset_index().drop(['index'], axis=1)

    # Remove titles that are not in the Nasdaq-100 window range
    window_returns.dropna(axis=1, how='any', inplace=True)

    # Get the name of the columns
    titles = window_returns.columns.tolist()

    # Remove the first two element in indexNames (Dates, NDX Index) because I don't need them
    # mantaining NDX Index in tmp
    titles = titles[2:]

    return window_returns, titles

## Example: First 180 days

In [9]:
rolling_df, titles = get_window_returns(0)
print("In the first {} days, {} stocks will be taken from the index and analyzed.".format(ROLLING_WINDOW_SIZE, len(rolling_df.columns)))
rolling_df.head().style.applymap(color_negative_red)

In the first 180 days, 101 stocks will be taken from the index and analyzed.


,Dates,NDX Index,AAPL,ADBE,ADI,ADP,ADSK,AMAT,AMGN,AMZN,ATVI,AVGO,BIDU,BIIB,BKNG,CHTR,CMCSA,COST,CSCO,CSX,CTAS,CTSH,DLTR,EA,EBAY,FAST,FB,FISV,GILD,GOOG,GOOGL,ILMN,INTC,INTU,ISRG,JD,KHC,KLAC,LRCX,MAR,MCHP,MDLZ,MNST,MSFT,MU,NFLX,NTES,NVDA,ORLY,PAYX,PCAR,PYPL,QCOM,REGN,ROST,SBUX,SIRI,SWKS,TMUS,TSLA,TXN,VRSK,VRTX,WBA,MAT,AKAM,DISCA,DISCK,NCLH,TSCO,VIAB,DISH,CA,XRAY,ESRX,HOLX,QRTEA,STX,SHPG,VOD,TFCF,TFCFA,CELG,HAS,HSIC,VTRS,NLOK,AAL,WDC,BMRN,CTXS,EXPE,LBTYA,ULTA,ALXN,MXIM,CERN,CHKP,INCY,TCOM,XLNX
0,4/1/2017,0.005256,-0.001137,0.006358,-0.002002,0.001545,0.017437,0.009349,0.014098,0.004646,0.019460,-0.007147,0.021514,0.007992,0.006557,0.020024,0.011805,0.000188,-0.014512,0.020412,0.008475,0.014055,0.025495,0.018273,-0.002685,0.004675,0.015538,0.012130,0.029496,0.000966,-0.000297,0.030418,-0.005205,0.002002,-0.019356,0.001161,-0.005729,-0.003532,0.000559,-0.001941,-0.000470,-0.001115,0.010485,-0.004484,-0.008461,0.014948,0.002472,0.023063,0.004583,0.003760,0.013684,0.018462,0.001070,0.008941,0.014225,0.011496,0.020068,-0.002131,0.008536,0.045055,-0.001224,0.007839,0.028434,0.000241,0.027019,0.011185,0.011733,0.009455,0.045234,0.011658,0.019311,0.011659,0.015976,0.008155,0.024763,-0.002993,0.010947,-0.012700,0.021027,0.022580,0.008889,0.010392,-0.001686,0.014185,0.003968,0.013208,0.008708,0.008602,0.015861,0.011928,0.001552,0.011495,0.022582,0.016384,0.035881,0.010560,-0.030209,0.004130,0.005069,0.020776,-0.007306
1,5/1/2017,0.005603,0.005090,0.016854,-0.014477,-0.005999,-0.007640,-0.004352,0.000719,0.030270,0.015405,-0.015882,0.031540,-0.001633,0.010973,0.015722,0.003999,0.019525,0.002323,0.009780,-0.008819,-0.004021,-0.017650,-0.020929,0.008365,-0.011728,0.016544,-0.000368,-0.004856,0.009007,0.006478,-0.004597,-0.001649,0.006156,0.005418,0.017258,-0.005762,-0.009013,-0.002239,-0.006824,-0.019461,0.004454,0.003687,0.000000,-0.011244,0.018376,0.046938,-0.025713,0.002478,-0.007205,-0.016444,0.001462,0.001221,-0.005733,-0.013007,0.008359,0.023992,-0.005616,-0.020785,-0.001058,-0.007786,-0.003667,0.025752,0.000602,0.055828,0.017697,-0.039291,-0.037991,-0.004926,-0.015221,0.023759,0.006108,0.008196,-0.013114,-0.003803,-0.006014,-0.023537,-0.001791,0.012495,0.029715,0.003181,-0.000689,0.001770,0.033650,-0.005376,-0.011931,0.017598,-0.017497,0.000142,-0.006699,-0.000333,0.008318,0.030472,0.012265,0.090715,-0.015232,-0.005859,0.020283,0.018975,0.034708,-0.012182
2,6/1/2017,0.008450,0.011086,0.022315,0.003918,0.000679,0.030342,-0.001871,0.024536,0.019716,-0.000791,0.013167,-0.006161,0.004859,0.011157,0.006697,0.001709,-0.000491,0.001987,0.014229,0.003605,0.013570,-0.012504,0.000633,0.034068,-0.002362,0.022453,0.011170,-0.006865,0.015161,0.014882,0.050219,0.003570,0.010062,0.005050,-0.001141,-0.002546,0.009013,0.004844,0.006338,0.017265,0.001332,-0.012417,0.008630,-0.003171,-0.005630,-0.009865,0.013279,-0.003649,0.006879,0.011296,0.009453,-0.000305,-0.060159,-0.005802,0.011797,-0.017392,0.005216,-0.014689,0.009918,0.016727,0.008778,0.004165,0.000843,-0.015307,0.003445,0.010241,0.009727,-0.006757,-0.006234,0.008237,0.009826,0.003320,-0.000686,-0.001695,0.002259,-0.004065,-0.014444,-0.010439,-0.002665,0.017836,0.017772,0.007383,-0.010085,0.003010,-0.016998,0.014499,0.006949,-0.002413,0.006469,0.008497,0.015229,0.031072,-0.003185,0.031333,0.033260,-0.002101,0.006671,0.032802,-0.007034,0.019149
3,9/1/2017,0.003553,0.009118,0.002490,0.004737,-0.006226,0.003650,0.022833,0.013054,0.001168,-0.005555,0.002150,0.004413,0.013536,0.005947,-0.006393,0.007938,-0.011489,-0.001655,-0.014229,-0.010509,-0.022544,0.000648,-0.004694,-0.009709,-0.007337,0.012001,-0.015877,0.004626,0.000620,0.002384,0.000353,0.003557,-0.003000,0.008131,-0.000381,-0.002785,0.005796,0.011642,-0.000608,0.011554,-0.008245,-0.027107,-0.003188,0.013520,-0.000916,0.034386,0.039743,-0.008482,-0.004089,-0.012371,-0.001207,0.001830,-0.010144,0.002141,0.018556,0.008734,0.009163,-0.001763,0.009863,0.002559,-0.007922,0.042780,-0.006641,0.001640,0.003005,-0.022448,-0.022972,0.003159,0.002790,0.01

# TITOLO GENERALE: TO REORDER
- returns calculator
- portfolio ranking
- portfolio builder

## Portfolio Implementation

AGGIUNGERE DESCRIZIONE

- TODO: RITORNARE ANCHE rank_df per fare dopo eventuali analisi nel report

In [10]:
def portfolio_ranked(selector, days):
    """
    Build a portfolio, based on the selector, by taking the titles included in the index in the range [days; days + ROLLING_WINDOW_SIZE]
    :param selector: The selector to use in order to build the rank and select the titles
    :param selector_columns: Optional values to use in the specific selector
    :param days: The number of days to skip 
    """
    
    rolling_df, titles = get_window_returns(days)

    rank_df = pd.DataFrame(columns=['Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns', 'systematic_risk'])

    ndx_returns = rolling_df.iloc[:, 1].values

    for title in titles:
        title_returns = rolling_df.iloc[0 : ROLLING_WINDOW_SIZE, rolling_df.columns.get_loc(title)]

        ndx_returns = sm.add_constant(ndx_returns)
        model = sm.OLS(title_returns, ndx_returns)
        result = model.fit()

        rank_df = rank_df.append({'Title': title, 'r2': result.rsquared, 'specific_risk': result.resid.std(), 'beta': result.params[1], 'alpha': result.params[0], 'alpha_significance': result.pvalues[0], 'absolute_returns': np.sum(title_returns),  'systematic_risk': result.params[1] ** 2 * ndx_returns.std() ** 2}, ignore_index=True)

    # Do the rank
    full_rank_df = rank_df.copy()
    if selector == 'max_r2':
        winners = rank_df.sort_values(by='r2', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_specific_risk':
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'absolute_returns':
        winners = rank_df.sort_values(by='absolute_returns', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_beta':
        winners = rank_df.sort_values(by='beta', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha':
        winners = rank_df.sort_values(by='alpha', ascending=True).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_r2_and_high_specific_risk':
        rank_df = rank_df.sort_values(by=['r2'], ascending=True)
        rank_df = rank_df.head(int(len(titles) * 1/3))
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_beta':
        winners = rank_df.sort_values(by='beta', ascending=True).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'high_systematic_risk' or selector == 'low_systematic_risk':
        ascending = False
        if 'low' in selector:
            ascending = True
        winners = rank_df.sort_values(by='systematic_risk', ascending=ascending).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_and_significant_alpha':
        rank_df = rank_df[rank_df['alpha_significance'] < 0.05]
        rank_df = rank_df[rank_df['alpha'] > 0]
        if(rank_df.shape[1] < 10):
            raise Exception("Not enought titles")
        winners = rank_df.sort_values(by='alpha', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha_and_high_beta':
        rank_df = rank_df[rank_df['alpha'] > 0]
        winners = rank_df.sort_values(by='beta', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    
    return selected_titles, winners, full_rank_df

In [11]:
def get_weekly_portfolio_returns(window_returns):
    
    returns = []
    left_the_market = [] # Keep here title already out of the portfolio

    for index, row in window_returns.iterrows():

        portfolio_components_number = 10
        daily_return = 0

        for title, value in row.items():
            if math.isnan(value):
                if title in left_the_market:
                    portfolio_components_number -= 1 # Skip this title. Removed from the market and from the portfolio
                else:
                    missing_date = nasdaq100_returns.iloc[index - 1]['Dates'] # Lo slicing di pandas sugli object ritorna l'indice + 1
                    missing_price = closing_prices_removed_titles.iloc[index][title]

                    if math.isnan(missing_price):
                        print(f'{title} is Nan at row {index} ({missing_date}). It was removed')

                        if title in title_out_of_the_market.keys():
                            missing_price = title_out_of_the_market[title]
                            last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                            missing_return = np.log(missing_price) - np.log(last_closing_price)
                            daily_return += missing_return
                        else:
                            raise Exception(f'Cannot find the closing/acquisition price of {title}')   

                        left_the_market.append(title)
                    else:
                        last_closing_price = closing_prices_removed_titles.iloc[index - 1][title]
                        missing_return = np.log(missing_price) - np.log(last_closing_price)
                        daily_return += missing_return
            else:
                 daily_return += value
            
        returns.append(daily_return / 10)
        #returns.append(daily_return / portfolio_components_number)
            
    return returns


In [12]:
def portfolio_builder(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1,11)] + ['Returns'])
    portfolio_history = pd.DataFrame(columns=['Dates', 'Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns'])

    titles, values, _ = portfolio_ranked(selector, 0)
    portfolio_history = portfolio_history.append(values, ignore_index=True)
       
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):

        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]

        tmp_returns = get_weekly_portfolio_returns(nasdaq100_window_returns)
        portfolio_returns = portfolio_returns + tmp_returns

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1,11)})
        portfolio = portfolio.append(portfolio_row, ignore_index=True)

        titles, values, _ = portfolio_ranked(selector, days)
        portfolio_history = portfolio_history.append(values, ignore_index=True)

    portfolio_history = portfolio_history[:-10]
    dates = np.array(portfolio['Dates'])
    dates = np.repeat(dates, 10)
    portfolio_history['Dates'] = dates
    return portfolio, portfolio_returns, portfolio_history
    

## Portfolio Selectors
- max_r2: desc..

DESCRIVERE I SELETTORI
E POI DARE UNA DESCRIZIONE A TUTTI I VARI DICTIONARY CHE CREIAMO

In [13]:
selectors = [
    'max_r2',
    'absolute_returns',
    'min_r2_and_high_specific_risk', 
    'max_specific_risk',
    'max_beta',
    'min_beta', 
    'positive_alpha',
    'high_systematic_risk',
    'low_systematic_risk',
    'positive_and_significant_alpha',
    'positive_alpha_and_high_beta'
]

selectors = []

In [14]:
base_metrics = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'MDD', 'CL', 'Var 90', 'Var 95', 'Var 99', 'IR', 'M2'])

In [15]:
portfolios_analysis = {}
days_limit = 0
for i in range(7, nasdaq100_returns.shape[0] - 180, 7):
    days_limit = i
ndx_returns = nasdaq100_returns[180: days_limit + 180]['NDX Index'].tolist()

In [16]:
portfolios_selector_history = {}

In [17]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')        

        portfolio, returns, history = portfolio_builder(selector)
        
        portfolios_selector_history[selector] = history
        portfolios_analysis[selector] = returns

        basic_row = get_base_metrics(selector, returns)
        base_metrics = base_metrics.append(basic_row, ignore_index=True)
        advanced_row = get_advanced_metrics(selector, returns, ndx_returns, risk_free_rate)
        advanced_metrics = advanced_metrics.append(advanced_row, ignore_index=True)
        
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


In [18]:
print('Remember that we\'re using log returns.')
print(tabulate(base_metrics, headers='keys', tablefmt='psql')) 

Remember that we're using log returns.
+-------------------+----------------------+-------------------------+
| Portfolio Title   | Annualized Returns   | Annualized Volatility   |
|-------------------+----------------------+-------------------------|
+-------------------+----------------------+-------------------------+


## ADVANCED METRICS
TO FIX VAR
- https://blog.quantinsti.com/calculating-value-at-risk-in-excel-python/
- https://www.interviewqs.com/blog/value-at-risk#:~:text=Value%20at%20risk%20(VaR)%20is,it%20across%20various%20confidence%20levels.

In [19]:
print(tabulate(advanced_metrics, headers='keys', tablefmt='psql')) 

+-------------------+----------------+-------+------+----------+----------+----------+------+------+
| Portfolio Title   | Sharpe Ratio   | MDD   | CL   | Var 90   | Var 95   | Var 99   | IR   | M2   |
|-------------------+----------------+-------+------+----------+----------+----------+------+------|
+-------------------+----------------+-------+------+----------+----------+----------+------+------+


# DO SOME CHARTS
TODO

# Non Compulsoty Tasks
- Momentum
- Simple Returns vs Log Returns
- A different weighting schema

In [20]:
selectors = ['max_r2']

In [21]:
def get_clenow_momentum(title, days):
    p = closing_prices[title].iloc[days: days + ROLLING_WINDOW_SIZE]
    p_log = np.log(p)
    x = np.arange(len(p_log)) 
    slope, _, rvalue, _, _ = linregress(x, p_log)
    m = ((1 + slope) ** 252) * (rvalue ** 2)
    return m

In [22]:
def get_window_returns_with_momentum(days):
    window_returns = nasdaq100_returns.iloc[days: days + ROLLING_WINDOW_SIZE]
    window_returns = window_returns.reset_index().drop(['index'], axis=1)

    # Remove titles that are not in the Nasdaq-100 window range
    window_returns.dropna(axis=1, how='any', inplace=True)

    # Get the name of the columns
    titles = window_returns.columns.tolist()

    # Remove the first two element in indexNames (Dates, NDX Index) because I don't need them
    # mantaining NDX Index in tmp
    titles = titles[2:]


    momentums = pd.DataFrame(columns=['Title', 'Momentum'])
    momentums['Title'] = titles

    for index, tt in momentums.iterrows():
        momentums.at[index,'Momentum'] = get_clenow_momentum(tt.Title, days)

    # Get first 1/3 titles with the highest momentum
    momentums = momentums.sort_values(by='Momentum', ascending=False)
    momentums = momentums.iloc[:int(len(momentums) / 3)]
    momentums = momentums['Title'].tolist()


    return window_returns, momentums

In [23]:
def portfolio_ranked_with_momentum(selector, days):
    """
    Build a portfolio, based on the selector, by taking the titles included in the index in the range [days; days + ROLLING_WINDOW_SIZE]
    :param selector: The selector to use in order to build the rank and select the titles
    :param selector_columns: Optional values to use in the specific selector
    :param days: The number of days to skip 
    """
    
    rolling_df, titles = get_window_returns_with_momentum(days)

    rank_df = pd.DataFrame(columns=['Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns', 'systematic_risk'])

    ndx_returns = rolling_df.iloc[:, 1].values

    for title in titles:
        title_returns = rolling_df.iloc[0 : ROLLING_WINDOW_SIZE, rolling_df.columns.get_loc(title)]

        ndx_returns = sm.add_constant(ndx_returns)
        model = sm.OLS(title_returns, ndx_returns)
        result = model.fit()

        rank_df = rank_df.append({'Title': title, 'r2': result.rsquared, 'specific_risk': result.resid.std(), 'beta': result.params[1], 'alpha': result.params[0], 'alpha_significance': result.pvalues[0], 'absolute_returns': np.sum(title_returns),  'systematic_risk': result.params[1] ** 2 * ndx_returns.std() ** 2}, ignore_index=True)

    # Do the rank
    full_rank_df = rank_df.copy()
    if selector == 'max_r2':
        winners = rank_df.sort_values(by='r2', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_specific_risk':
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'absolute_returns':
        winners = rank_df.sort_values(by='absolute_returns', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'max_beta':
        winners = rank_df.sort_values(by='beta', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha':
        winners = rank_df.sort_values(by='alpha', ascending=True).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_r2_and_high_specific_risk':
        rank_df = rank_df.sort_values(by=['r2'], ascending=True)
        rank_df = rank_df.head(int(len(titles) * 1/3))
        winners = rank_df.sort_values(by='specific_risk', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'min_beta':
        winners = rank_df.sort_values(by='beta', ascending=True).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'high_systematic_risk' or selector == 'low_systematic_risk':
        ascending = False
        if 'low' in selector:
            ascending = True
        winners = rank_df.sort_values(by='systematic_risk', ascending=ascending).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_and_significant_alpha':
        rank_df = rank_df[rank_df['alpha_significance'] < 0.05]
        rank_df = rank_df[rank_df['alpha'] > 0]
        if(rank_df.shape[1] < 10):
            raise Exception("Not enought titles")
        winners = rank_df.sort_values(by='alpha', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    elif selector == 'positive_alpha_and_high_beta':
        rank_df = rank_df[rank_df['alpha'] > 0]
        winners = rank_df.sort_values(by='beta', ascending=False).head(10)
        selected_titles = winners['Title'].tolist()
    
    return selected_titles, winners, full_rank_df

In [24]:
def portfolio_builder_with_momentum(selector):
    portfolio_returns = []
    portfolio = pd.DataFrame(columns=['Dates'] + [f'Title{i}' for i in range(1,11)] + ['Returns'])
    portfolio_history = pd.DataFrame(columns=['Dates', 'Title', 'r2', 'specific_risk', 'beta', 'alpha', 'alpha_significance', 'absolute_returns'])

    titles, values, _ = portfolio_ranked_with_momentum(selector, 0)
    portfolio_history = portfolio_history.append(values, ignore_index=True)
       
    days_range = nasdaq100_returns.shape[0] - ROLLING_WINDOW_SIZE
    for days in range(7, days_range, 7):

        nasdaq100_window_returns = nasdaq100_returns[titles].iloc[ROLLING_WINDOW_SIZE + days - 7 : ROLLING_WINDOW_SIZE + days]

        tmp_returns = get_weekly_portfolio_returns(nasdaq100_window_returns)
        portfolio_returns = portfolio_returns + tmp_returns

        portfolio_row = {'Dates': nasdaq100_returns.iloc[ROLLING_WINDOW_SIZE + days]['Dates']}
        portfolio_row.update({f'Title{i}': titles[i - 1] for i in range(1,11)})
        portfolio = portfolio.append(portfolio_row, ignore_index=True)

        titles, values, _ = portfolio_ranked_with_momentum(selector, days)
        portfolio_history = portfolio_history.append(values, ignore_index=True)

    portfolio_history = portfolio_history[:-10]
    dates = np.array(portfolio['Dates'])
    dates = np.repeat(dates, 10)
    portfolio_history['Dates'] = dates
    return portfolio, portfolio_returns, portfolio_history
    

In [25]:
base_metrics_with_momentum = pd.DataFrame(columns=['Portfolio Title', 'Annualized Returns', 'Annualized Volatility'])
advanced_metrics_with_momentum = pd.DataFrame(columns=['Portfolio Title', 'Sharpe Ratio', 'MDD', 'CL', 'Var 90', 'Var 95', 'Var 99', 'IR', 'M2'])

In [26]:
portfolios_selector_history_with_momentum = {}

In [27]:
for selector in selectors:
    try:
        print(f'Buildindg {selector} portfolio')        

        portfolio, returns, history = portfolio_builder_with_momentum(selector)
        
        # portfolios_selector_history[selector] = history
        # portfolios_analysis[selector] = returns

        basic_row = get_base_metrics(selector, returns)
        base_metrics_with_momentum = base_metrics_with_momentum.append(basic_row, ignore_index=True)
        advanced_row = get_advanced_metrics(selector, returns, ndx_returns, risk_free_rate)
        advanced_metrics_with_momentum = advanced_metrics_with_momentum.append(advanced_row, ignore_index=True)
        
    except Exception as e:
        print(f'Cannot build the portfolio for the selector {selector}: {str(e)}')


Buildindg max_r2 portfolio
MXIM is Nan at row 1170 (26/8/2021). It was removed
XLNX is Nan at row 1288 (14/2/2022). It was removed


In [28]:
print('Remember that we\'re using log returns.')
print(tabulate(base_metrics_with_momentum, headers='keys', tablefmt='psql')) 

Remember that we're using log returns.
+----+-------------------+----------------------+-------------------------+
|    | Portfolio Title   | Annualized Returns   | Annualized Volatility   |
|----+-------------------+----------------------+-------------------------|
|  0 | max_r2            | 13.38%               | 30.64%                  |
+----+-------------------+----------------------+-------------------------+


In [29]:
print('Remember that we\'re using log returns.')
print(tabulate(advanced_metrics_with_momentum, headers='keys', tablefmt='psql')) 

Remember that we're using log returns.
+----+-------------------+----------------+-----------+----------+------------+------------+------------+-----------+----------+
|    | Portfolio Title   |   Sharpe Ratio |       MDD |       CL |     Var 90 |     Var 95 |     Var 99 |        IR |       M2 |
|----+-------------------+----------------+-----------+----------+------------+------------+------------+-----------+----------|
|  0 | max_r2            |       0.402564 | -0.333106 | 0.401574 | -0.0242011 | -0.0312123 | -0.0443641 | -0.152581 | 0.111947 |
+----+-------------------+----------------+-----------+----------+------------+------------+------------+-----------+----------+
